# Distance measures

In [ ]:
import numpy as np
import forest.benchmarking.random_operators as rand_ops
import forest.benchmarking.distance_measures as dm

## Purity

In [ ]:
# make a random pure state and turn into a state matrix
psi = rand_ops.haar_rand_state(2)
rho_pure = np.matmul(psi,np.conjugate(psi.T))

# maximally mixed state
rho_max_mixed = np.eye(2)/2
rho_qutrit_max_mixed = np.eye(3)/3

# calculate purity without dimensional renormalization
print(dm.purity(rho_pure, dim_renorm=False))
print(dm.purity(rho_max_mixed, dim_renorm=False))
print(dm.purity(rho_qutrit_max_mixed, dim_renorm=False))
print("------------------")

# actually this is the default option
print(dm.purity(rho_pure))
print(dm.purity(rho_max_mixed))
print(dm.purity(rho_qutrit_max_mixed))

Now look at case with dimensional renormalization makes purity between [0,1].

The maximally mixed state has purity = 0, independent of dimension D.

In [ ]:
# calculate purity WITH dimensional renormalization
print(dm.purity(rho_pure, dim_renorm=True))
print(dm.purity(rho_max_mixed, dim_renorm=True))
print(dm.purity(rho_qutrit_max_mixed, dim_renorm=True))

## Distance measures between quantum states

In [ ]:
# random pure qubit state
psi = rand_ops.haar_rand_state(2)
rho = np.matmul(psi,np.conjugate(psi.T))

# random mixed qubit state
sigma = rand_ops.ginibre_state_matrix(2,2)

## fidelity

In [ ]:
dm.fidelity(rho, sigma)

## trace distance

In [ ]:
dm.trace_distance(rho, sigma)

### Bures distance

In [ ]:
dm.bures_distance(rho, sigma)

## Bures angle

In [ ]:
dm.bures_angle(rho, sigma)

### Hilbert Schmidt IP

In [ ]:
dm.hilbert_schmidt_ip(rho, sigma)

## Quantum chernoff exponent

In [ ]:
qcb_exp, s_opt = dm.quantum_chernoff_bound(rho,sigma)
print(qcb_exp, s_opt)

## Diamond norm AKA the completely bounded trace norm

In [ ]:
# Related to test cases borrowed from qutip,
# https://github.com/qutip/qutip/blob/master/qutip/tests/test_metrics.py
# which were in turn generated using QuantumUtils for MATLAB
# (https://goo.gl/oWXhO9) by C. Granade

Id = np.asarray([[1, 0], [0, 1]])
Xd = np.asarray([[0, 1], [1, 0]])

from scipy.linalg import expm
# Define unitary
theta = 0.4
Ud = expm(-theta*1j*Xd/2)
# = cos(theta)Id + 1j sin(theta)X
# This unitary is:
# close to Id for theta small
# close to X for theta np.pi (up to global phase -1j)
print(Ud)

In [ ]:
def _gate_to_superop(gate):
    dim = gate.shape[0]
    superop = np.outer(gate, gate.conj().T)
    superop = np.reshape(superop, [dim]*4)
    superop = np.transpose(superop, [0, 3, 1, 2])
    return superop

def _superop_to_choi(superop):
    dim = superop.shape[0]
    superop = np.transpose(superop, (0, 2, 1, 3))
    choi = np.reshape(superop, [dim**2] * 2)
    return choi

def _gate_to_choi(gate):
    return _superop_to_choi(_gate_to_superop(gate))

In [ ]:
choi0 = _gate_to_choi(Id) 
choi1 = _gate_to_choi(Ud)
choi2 = _gate_to_choi(Xd)

In [ ]:
dnorm = dm.diamond_norm(choi0, choi1)
print("This gate is close to the identity as the diamond norm is close to zero. Dnorm= ",dnorm)

In [ ]:
dnorm = dm.diamond_norm(choi0, choi2)
print("This gate is far from identity as diamond norm = ",dnorm)